In [5]:
from datetime import datetime

# Configurações gerais do grid (Bybit)
simbolos = ['BTCUSDT', 'SOLUSDT', 'XRPUSDT', 'ADAUSDT', 'TRXUSDT'] #ETHUSDT, BNBUSDT removidos por resultados ruins
intervalos = ['15']
data_inicio = '2023-01-01' #Testado 2023 até agora. Testar 2022 até agora e 2023 até 2025.
# data_fim = '2024-12-31'
data_fim = datetime.now().strftime('%Y-%m-%d')

# Parâmetros do grid - possível separar a geração de combinações em utils/grid_generator.py
grid_ema_curta = list(range(5, 50, 3))
grid_ema_longa = list(range(15, 151, 5))
grid_stop = list(range(9, 21, 1))
grid_rr = [round(x, 1) for x in list([1.5 + 0.1 * i for i in range(30)])]  # from 1.5 to 4.5

In [ ]:
import sys
import os
# Adiciona o diretório raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..', 'src')))
from vectorbt_project.utils.telegram_compatibility import apply_vectorbt_telegram_patch
apply_vectorbt_telegram_patch()
import time
from itertools import product
import psutil
import numpy as np
import vectorbt as vbt
import pandas as pd
from corretoras.funcoes_bybit import carregar_dados_historicos
from entidades.estrategias_descritivas.double_ema_breakout import DoubleEmaBreakout, CondicaoEntrada, StopConfig, AlvoConfig
from vectorbt_project.generator_vectorbt import gerar_por_nome

# Garantir pasta de resultados
os.makedirs("data/results", exist_ok=True)

for simbolo in simbolos:
    for intervalo in intervalos:
        df = carregar_dados_historicos(simbolo, intervalo, [9, 21], data_inicio, data_fim)
        df.columns = df.columns.str.lower()

        # Executar todas as combinações
        resultados = []
        # equity_curves = {}

        total = len([
            (c, l, s, r)
            for c, l, s, r in product(grid_ema_curta, grid_ema_longa, grid_stop, grid_rr)
            if l > c
        ])
        print(f"Executando {total} combinações...")

        testados = 0
        start = time.time()

        for ema_curta in grid_ema_curta:
            for ema_longa in grid_ema_longa:
                if ema_longa <= ema_curta:
                    continue  # evita combinações incoerentes
                for stop in grid_stop:
                    for rr in grid_rr:
                        percent = (testados + 1) / total * 100
                        testados += 1
                        bar = '█' * int(percent / 2) + '-' * (50 - int(percent / 2))
                        elapsed = time.time() - start
                        tempo_medio = elapsed / testados
                        estimado_restante = (total - testados) * tempo_medio
                        estimado_str = f"{int(estimado_restante // 60)}m {int(estimado_restante % 60)}s"

                        print(f"\r[{bar}] {percent:.2f}% - {testados+1}/{total} | "
                        f"Decorrido: {int(elapsed // 60)}m {int(elapsed % 60)}s | "
                        f"Estimado: {estimado_str} | "
                        f"[RAM] {psutil.virtual_memory().percent}% usada", end='', flush=True)

                        nome = f"ema_{ema_curta}_{ema_longa}_stop_{stop}_rr_{rr}"
                        estrategia = DoubleEmaBreakout(
                            nome=nome,
                            tipo="long",
                            condicoes_entrada=[
                                CondicaoEntrada(tipo="fechamento_acima_ema", parametros={"periodo": ema_curta}),
                                CondicaoEntrada(tipo="fechamento_acima_ema", parametros={"periodo": ema_longa}),
                                CondicaoEntrada(tipo="rompe_maxima_anterior", parametros={})
                            ],
                            stop=StopConfig(tipo="minima_das_ultimas", parametros={"quantidade": stop}),
                            alvo=AlvoConfig(tipo="rr", parametros={"multiplicador": rr})
                        )

                        try:
                            # entries, exits, stop_price, target_price = gerar_por_nome("double_ema_breakout", df, estrategia)
                            entries, exits, _, _ = gerar_por_nome("double_ema_breakout_orders", df, estrategia)

                            size = pd.Series(0, index=df.index, dtype='float64')
                            size[entries.notna()] = 1.0  # Compra (1)
                            size[exits.notna()] = -1.0   # Venda (-1)
                            size = size.where(size != 0, np.nan)

                            order_price = entries.combine_first(exits)
                            
                            # Criar portfólio com sinais e preços de execução
                            pf = vbt.Portfolio.from_orders(
                                close=df['fechamento'],
                                price=order_price,
                                size_type='percent',
                                size=size,
                                init_cash=1000,
                                fees=0.00055,  # Taxa de corretagem
                                # slippage=0.0001,  # Slippage (diferença entre o preço de entrada e saída)
                                freq=f'{intervalo}min',  # Frequência de 15 minutos
                                direction='longonly'
                            )
                            stats = pf.stats()

                            resultados.append({
                                "moeda": simbolo,
                                "intervalo": intervalo,
                                "periodo": f"{data_inicio} : {data_fim}",
                                "estrategia": nome,
                                "ema_curta": ema_curta,
                                "ema_longa": ema_longa,
                                "stop": stop,
                                "rr": rr,
                                "saldo_inicial": 1000,
                                "saldo_final": stats['End Value'],
                                "fitness": stats['Total Return [%]'] * (1 - stats['Max Drawdown [%]'] / 100),
                                "retorno_total": stats['Total Return [%]'],
                                "max_drawdown": stats['Max Drawdown [%]'],
                                "max_drawdown_duration": stats['Max Drawdown Duration'],
                                "trades": stats['Total Trades'],
                                "win_rate": stats['Win Rate [%]'],
                                "ganho_medio": stats['Avg Winning Trade [%]'],
                                "perda_media": stats['Avg Losing Trade [%]'],
                                'melhor_trade': stats['Best Trade [%]'],
                                'pior_trade': stats['Worst Trade [%]'],
                                "sharpe_ratio": stats['Sharpe Ratio'],
                                "sortino_ratio": stats['Sortino Ratio'],
                                "calmar_ratio": stats['Calmar Ratio'],
                            })

                            # equity_curves[nome] = pf

                        except Exception as e:
                            print(f"Erro em {nome}: {e}")

        # Criar subpastas de resultados, se ainda não existirem
        os.makedirs("data/results/grids", exist_ok=True)
        os.makedirs("data/results/tops", exist_ok=True)
        # os.makedirs("data/results/equities", exist_ok=True)

        # Salvar resultados por ordem de retorno_total e menor max_drawdown
        df_resultados = pd.DataFrame(resultados).sort_values(by="fitness", ascending=False)
        # df_drawdown = pd.DataFrame(resultados).sort_values(by="max_drawdown", ascending=True)

        now = datetime.now().strftime("%Y%m%d_%H%M%S")
        caminho_csv = f"data/results/grids/{now}_double_ema_breakout_orders_{simbolo}_{intervalo}_{data_inicio}_{data_fim}.csv"
        df_resultados.to_csv(caminho_csv, index=False)
        # caminho_csv_drawdown = f"data/results/grids/double_ema_breakout_drawdown_{now}.csv"
        # df_drawdown.to_csv(caminho_csv_drawdown, index=False)
        # print(f"\n✅ Resultados salvos em {caminho_csv} e {caminho_csv_drawdown}")

        print(f"\n🏁 Grid search concluído:")
        nro_resultados = 100
        top_resultados = df_resultados.head(nro_resultados).sort_values(by="retorno_total", ascending=False)
        print(top_resultados)

        # Exportar top resultados como JSON
        caminho_json = f"data/results/tops/{now}_top{nro_resultados}_double_ema_breakout_orders_{simbolo}_{intervalo}_{data_inicio}_{data_fim}.json"
        top_resultados.to_json(caminho_json, orient="records", indent=4)

        # Exibir resultados
        print("\n✅ Execução concluída!")
        print(f"\n📊 Resultados salvos em:")
        print(f"   - CSV: {caminho_csv}")
        print(f"   - JSON: {caminho_json}")

Carregando dados históricos...
Velas carregadas do arquivo: data/dados_historicos\BTCUSDT_15m_2023-01-01-2025-05-04.json
Executando 164160 combinações...
[██████████████████████████████████████████████████] 100.00% - 164161/164160 | Decorrido: 162m 37s | Estimado: 0m 0s | [RAM] 47.3% usadaa
🏁 Grid search concluído:
          moeda intervalo                  periodo                estrategia  \
3884    BTCUSDT        15  2023-01-01 : 2025-05-04   ema_5_55_stop_18_rr_2.9   
52484   BTCUSDT        15  2023-01-01 : 2025-05-04  ema_17_55_stop_18_rr_2.9   
16124   BTCUSDT        15  2023-01-01 : 2025-05-04   ema_8_55_stop_18_rr_2.9   
38096   BTCUSDT        15  2023-01-01 : 2025-05-04  ema_14_27_stop_18_rr_4.1   
49976   BTCUSDT        15  2023-01-01 : 2025-05-04  ema_17_27_stop_18_rr_4.1   
...         ...       ...                      ...                       ...   
146317  BTCUSDT        15  2023-01-01 : 2025-05-04  ema_44_51_stop_14_rr_2.2   
85929   BTCUSDT        15  2023-01-01 : 202